In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row
import random

In [0]:
# Sample Indian names and cities
names = ["Aarav", "Vihaan", "Vivaan", "Ananya", "Ishaan", "Kabir", "Aanya", "Myra", "Reyansh", "Siya", "Neha", "Raj", "Aditi", "Aryan", "Priya", "Rohan", "Sanya", "Riya", "Kunal", "Deepika", "Yash", "Rishi", "Sarthak", "Anaya", "Tanvi", "Pooja", "Varun", "Siddharth", "Harsh", "Gaurav", "Swati", "Manish", "Rakesh", "Nitin", "Suman", "Akash", "Suhas", "Rahul", "Bhavana", "Sandeep", "Kiran", "Preeti", "Vikram", "Tanya", "Nikhil", "Chetan", "Madhuri", "Sonali", "Pankaj", "Meera"]
cities = ["Mumbai", "Delhi", "Bangalore", "Hyderabad", "Chennai", "Kolkata", "Pune", "Ahmedabad", "Jaipur", "Lucknow"]

# Generate 50 employee records
employees = [
    Row(emp_id=i+1, 
        emp_name=random.choice(names), 
        emp_city=random.choice(cities), 
        emp_salary=random.randint(30000, 150000))
    for i in range(50)
]

In [0]:
# Define schema
schema = StructType([
    StructField("emp_id", IntegerType(), False),
    StructField("emp_name", StringType(), False),
    StructField("emp_city", StringType(), False),
    StructField("emp_salary", IntegerType(), False)
])

# Create DataFrame
emp_df = spark.createDataFrame(employees, schema)

In [0]:
display(emp_df)

emp_id,emp_name,emp_city,emp_salary
1,Pooja,Chennai,45526
2,Aarav,Kolkata,137256
3,Vikram,Ahmedabad,51001
4,Varun,Bangalore,108744
5,Pooja,Kolkata,88118
6,Madhuri,Pune,136703
7,Vihaan,Ahmedabad,114558
8,Madhuri,Kolkata,44678
9,Kiran,Lucknow,110727
10,Reyansh,Jaipur,40533


In [0]:
emp_df = emp_df.repartition(5)
emp_df.rdd.getNumPartitions()

Out[5]: 5

In [0]:
emp_df = emp_df.coalesce(10)
emp_df.rdd.getNumPartitions()

Out[6]: 5

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/student', recurse=True)

Out[20]: True

In [0]:
%sql
CREATE TABLE STUDENT (id INT, NAME VARCHAR(20), CITY VARCHAR(10))
USING DELTA;

In [0]:
# %sql
# ALTER TABLE STUDENT 
# ADD COLUMNS CITY VARCHAR(20);

In [0]:
%sql
INSERT INTO student (ID, NAME, CITY)
VALUES 
(1, 'BHIM', 'DHOLAKPUR'), 
(2, 'CHUTKI', 'DHOKALPUR'), 
(3, 'KALIA', 'BOMBAY'), 
(4, 'JAGGU', 'JUNGLE'),
(5, 'RAJU', 'BOMBAY');


num_affected_rows,num_inserted_rows
5,5


In [0]:
# %sql
# UPDATE STUDENT
# SET CITY = 'DHOLAKPUR'
# WHERE NAME IN ('BHIM', 'CHUTKI');

In [0]:
# %sql
# UPDATE STUDENT
# SET CITY = 'MAYANAGAR'
# WHERE NAME IN ('KALIA');

In [0]:
# %sql
# UPDATE STUDENT
# SET CITY = 'KATHMANDU'
# WHERE NAME = 'RAJU';

In [0]:
%sql
DESCRIBE EXTENDED STUDENT;

col_name,data_type,comment
id,int,null
NAME,varchar(20),null
CITY,varchar(10),null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,default,
Table,STUDENT,
Created Time,Sat Mar 08 14:47:57 UTC 2025,
Last Access,UNKNOWN,


In [0]:
from delta.tables import *
from pyspark.sql.functions import *

In [0]:
delta_table = DeltaTable.forName(spark, "Student")
delta_table.delete("NAME == 'RAJU'")

In [0]:
delta_path = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/student')
display(delta_path)

In [0]:
df = spark.sql('SELECT * FROM STUDENT')
df = df.repartition(5)
df.rdd.getNumPartitions()

Out[42]: 5

In [0]:
df = df.coalesce(10)
df.rdd.getNumPartitions()

Out[43]: 5

UPSERT means Update or Insert

In [0]:
%sql
select * from student;

id,NAME,CITY
1,BHIM,DHOLAKPUR
2,CHUTKI,DHOKALPUR
3,KALIA,BOMBAY
4,JAGGU,JUNGLE


In [0]:
%sql
describe extended student_new 

col_name,data_type,comment
id,int,null
name,varchar(20),null
city,varchar(10),null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,default,
Table,student_new,
Created Time,Sat Mar 08 15:36:53 UTC 2025,
Last Access,UNKNOWN,


In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/student_new', recurse=True)

Out[53]: False

In [0]:
%sql
create table student_new (id INT, name varchar(20), city varchar(10)) using delta;

In [0]:
%sql
insert into student_new 
(id, name, city) 
values 
(1, 'BHIM', 'dholakpur1'),
(5, 'RAJU', 'RajukaGhar');

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
SELECT * from student_new;

id,name,city
1,BHIM,dholakpur1
5,RAJU,RajukaGhar


## SCD 1 Implementaion

In [0]:
%sql
MERGE INTO student_new as sn
USING student as s 
ON sn.id = s.id
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT (
    id, 
    name, 
    city
  )
VALUES (
    s.id, 
    s.name, 
    s.city
  )

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
4,1,0,3


In [0]:
%sql
select * from student_new;

id,name,city
1,BHIM,DHOLAKPUR
2,CHUTKI,DHOKALPUR
3,KALIA,BOMBAY
4,JAGGU,JUNGLE
5,RAJU,RajukaGhar


In [0]:
%sql
select * from student;

id,NAME,CITY
1,BHIM,DHOLAKPUR
2,CHUTKI,DHOKALPUR
3,KALIA,BOMBAY
4,JAGGU,JUNGLE


In [0]:
%sql
DESCRIBE HISTORY student_new;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-03-08T15:48:13.000+0000,942438858102300,punit.prajapati@ridgeant.com,MERGE,"Map(predicate -> [""(id#5125 = id#5131)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2643570060988732),0308-143947-spir2od1,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 1, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetBytesAdded -> 2192, numTargetBytesRemoved -> 1158, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 1, executionTimeMs -> 7186, materializeSourceTimeMs -> 17, numTargetRowsInserted -> 3, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 3308, numTargetRowsUpdated -> 1, numOutputRows -> 5, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 4, numTargetFilesRemoved -> 1, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 3454)",null,Databricks-Runtime/12.2.x-scala2.12
1,2025-03-08T15:44:11.000+0000,942438858102300,punit.prajapati@ridgeant.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2643570060988732),0308-143947-spir2od1,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 2, numOutputBytes -> 1158)",null,Databricks-Runtime/12.2.x-scala2.12
0,2025-03-08T15:39:54.000+0000,942438858102300,punit.prajapati@ridgeant.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2643570060988732),0308-143947-spir2od1,null,WriteSerializable,true,Map(),null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
%sql
CREATE VIEW st_v1 AS (
  SELECT * FROM student_new Version as of 1
);

SELECT * FROM st_v1;

id,name,city
1,BHIM,dholakpur1
5,RAJU,RajukaGhar
